# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 8:30AM,258394,10,0086463669,ISDIN Corporation,Released
1,Mar 1 2023 8:30AM,258394,10,0086463823,ISDIN Corporation,Released
2,Mar 1 2023 8:30AM,258394,10,0086462425,ISDIN Corporation,Released
3,Mar 1 2023 8:30AM,258394,10,0086462431,ISDIN Corporation,Released
4,Mar 1 2023 8:30AM,258394,10,0086462438,ISDIN Corporation,Released
5,Mar 1 2023 8:30AM,258394,10,0086462436,ISDIN Corporation,Released
6,Mar 1 2023 8:30AM,258394,10,0086463131,ISDIN Corporation,Released
7,Mar 1 2023 8:30AM,258394,10,0086463664,ISDIN Corporation,Released
8,Mar 1 2023 8:30AM,258394,10,0086462377,ISDIN Corporation,Released
9,Mar 1 2023 8:30AM,258394,10,0086462378,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,258402,Released,1
22,258404,Released,1
23,258406,Released,1
24,258407,Released,1
25,258408,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258402,NaN,NaN,1.0
258404,NaN,NaN,1.0
258406,NaN,NaN,1.0
258407,NaN,NaN,1.0
258408,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258330,0.0,1.0,0.0
258349,1.0,2.0,12.0
258351,0.0,0.0,1.0
258353,0.0,0.0,17.0
258354,0.0,0.0,49.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258330,0,1,0
258349,1,2,12
258351,0,0,1
258353,0,0,17
258354,0,0,49


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258330,0,1,0
1,258349,1,2,12
2,258351,0,0,1
3,258353,0,0,17
4,258354,0,0,49


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258330,,1,
1,258349,1,2,12
2,258351,,,1
3,258353,,,17
4,258354,,,49


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 8:30AM,258394,10,ISDIN Corporation
16,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc."
17,Mar 1 2023 8:30AM,258397,10,Bio-PRF
23,Mar 1 2023 8:30AM,258398,10,"Lupin Research, Inc."
24,Feb 28 2023 4:39PM,258408,18,"Emersa Waterbox, LLC"
25,Feb 28 2023 4:37PM,258407,18,"Emersa Waterbox, LLC"
26,Feb 28 2023 4:33PM,258406,18,"Emersa Waterbox, LLC"
27,Feb 28 2023 4:19PM,258404,22,"NBTY Global, Inc."
28,Feb 28 2023 3:05PM,258395,19,Eli Lilly and Company
29,Feb 28 2023 2:51PM,258393,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 1 2023 8:30AM,258394,10,ISDIN Corporation,,,16
1,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",,,1
2,Mar 1 2023 8:30AM,258397,10,Bio-PRF,,,6
3,Mar 1 2023 8:30AM,258398,10,"Lupin Research, Inc.",,,1
4,Feb 28 2023 4:39PM,258408,18,"Emersa Waterbox, LLC",,,1
5,Feb 28 2023 4:37PM,258407,18,"Emersa Waterbox, LLC",,,1
6,Feb 28 2023 4:33PM,258406,18,"Emersa Waterbox, LLC",,,1
7,Feb 28 2023 4:19PM,258404,22,"NBTY Global, Inc.",,,1
8,Feb 28 2023 3:05PM,258395,19,Eli Lilly and Company,,,1
9,Feb 28 2023 2:51PM,258393,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 8:30AM,258394,10,ISDIN Corporation,16,,
1,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",1,,
2,Mar 1 2023 8:30AM,258397,10,Bio-PRF,6,,
3,Mar 1 2023 8:30AM,258398,10,"Lupin Research, Inc.",1,,
4,Feb 28 2023 4:39PM,258408,18,"Emersa Waterbox, LLC",1,,
5,Feb 28 2023 4:37PM,258407,18,"Emersa Waterbox, LLC",1,,
6,Feb 28 2023 4:33PM,258406,18,"Emersa Waterbox, LLC",1,,
7,Feb 28 2023 4:19PM,258404,22,"NBTY Global, Inc.",1,,
8,Feb 28 2023 3:05PM,258395,19,Eli Lilly and Company,1,,
9,Feb 28 2023 2:51PM,258393,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 8:30AM,258394,10,ISDIN Corporation,16,,
1,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",1,,
2,Mar 1 2023 8:30AM,258397,10,Bio-PRF,6,,
3,Mar 1 2023 8:30AM,258398,10,"Lupin Research, Inc.",1,,
4,Feb 28 2023 4:39PM,258408,18,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 8:30AM,258394,10,ISDIN Corporation,16.0,NaN,NaN
1,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Mar 1 2023 8:30AM,258397,10,Bio-PRF,6.0,NaN,NaN
3,Mar 1 2023 8:30AM,258398,10,"Lupin Research, Inc.",1.0,NaN,NaN
4,Feb 28 2023 4:39PM,258408,18,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 8:30AM,258394,10,ISDIN Corporation,16.0,0.0,0.0
1,Mar 1 2023 8:30AM,258402,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Mar 1 2023 8:30AM,258397,10,Bio-PRF,6.0,0.0,0.0
3,Mar 1 2023 8:30AM,258398,10,"Lupin Research, Inc.",1.0,0.0,0.0
4,Feb 28 2023 4:39PM,258408,18,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4133985,159.0,2.0,1.0
18,775221,3.0,0.0,0.0
19,516725,1.0,1.0,0.0
22,775137,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4133985,159.0,2.0,1.0
1,18,775221,3.0,0.0,0.0
2,19,516725,1.0,1.0,0.0
3,22,775137,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,159.0,2.0,1.0
1,18,3.0,0.0,0.0
2,19,1.0,1.0,0.0
3,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,159.0
1,18,Released,3.0
2,19,Released,1.0
3,22,Released,3.0
4,10,Executing,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,18,19,22
Status,,,,
Completed,1.0,0.0,0.0,0.0
Executing,2.0,0.0,1.0,0.0
Released,159.0,3.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,18,19,22
0,Completed,1.0,0.0,0.0,0.0
1,Executing,2.0,0.0,1.0,0.0
2,Released,159.0,3.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,18,19,22
0,Completed,1.0,0.0,0.0,0.0
1,Executing,2.0,0.0,1.0,0.0
2,Released,159.0,3.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()